In [2]:
import pandas as pd
df_vax = pd.read_csv("COVID-19_Vaccinations.csv")

# Convert the 'Date' column to datetime format
df_vax["Date"] = pd.to_datetime(df_vax["Date"], format="%m/%d/%Y")

# Extract Year, Month, and Day
df_vax["Year"] = df_vax["Date"].dt.year
df_vax["Month"] = df_vax["Date"].dt.month
df_vax["Day"] = df_vax["Date"].dt.day

df_vax["Recip_County"] = df_vax["Recip_County"].str.replace(" County", "").str.strip()

# Rename columns to match indexing convention
df_vax = df_vax.rename(columns={"Recip_County": "County", "Recip_State": "State"})

# Select only relevant columns
df_vax = df_vax[["State", "County", "Year", "Month", "Day", "Series_Complete_Yes", 
                 "Series_Complete_Pop_Pct", "Booster_Doses_Vax_Pct"]]

# Set the multi-index
df_vax.set_index(["State", "County", "Year", "Month", "Day"], inplace=True)

# Display first few rows
print(df_vax.head())

# Display first few rows
print(df_vax.head())

/var/folders/dt/gkjsyh2n22vfkf1494tc58km0000gn/T/ipykernel_28834/2509695944.py:2: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_vax = pd.read_csv("COVID-19_Vaccinations.csv")


                               Series_Complete_Yes  Series_Complete_Pop_Pct  \
State County   Year Month Day                                                 
WI    Washburn 2023 5     10               10325.0                     65.7   
IA    Taylor   2023 5     10                2951.0                     48.2   
NY    Nassau   2023 5     10             1179481.0                     86.9   
TX    Lampasas 2023 5     10               10007.0                     46.7   
MI    Saginaw  2023 5     10              103137.0                     54.1   

                               Booster_Doses_Vax_Pct  
State County   Year Month Day                         
WI    Washburn 2023 5     10                    65.9  
IA    Taylor   2023 5     10                    61.1  
NY    Nassau   2023 5     10                    46.5  
TX    Lampasas 2023 5     10                    42.1  
MI    Saginaw  2023 5     10                    56.9  
                               Series_Complete_Yes  Series_Co

In [3]:
# Load cases dataset
df_cases = pd.read_csv("covid_confirmed_usafacts.csv", low_memory=False)

# Standardize county names (remove " County")
df_cases["County Name"] = df_cases["County Name"].str.replace(" County", "").str.strip()

# Rename columns for consistency
df_cases = df_cases.rename(columns={"County Name": "County", "State": "State"})

# Drop unnecessary columns
df_cases = df_cases.drop(columns=["countyFIPS", "StateFIPS"])

# Convert wide format (dates as columns) to long format
df_cases = df_cases.melt(id_vars=["State", "County"], var_name="Date", value_name="Cases")

# Convert Date column to datetime and extract Year, Month, Day
df_cases["Date"] = pd.to_datetime(df_cases["Date"])
df_cases["Year"] = df_cases["Date"].dt.year
df_cases["Month"] = df_cases["Date"].dt.month
df_cases["Day"] = df_cases["Date"].dt.day


# Drop the original Date column
df_cases = df_cases.drop(columns=["Date"])
df_cases = df_cases[df_cases["County"] != "Statewide Unallocated"]

# Set multi-index
df_cases.set_index(["State", "County", "Year", "Month", "Day"], inplace=True)

# Display first few rows
df_cases.head()


Cases
State County  Year Month Day       
AL    Autauga 2020 1     22       0
      Baldwin 2020 1     22       0
      Barbour 2020 1     22       0
      Bibb    2020 1     22       0
      Blount  2020 1     22       0

In [4]:
df_deaths = pd.read_csv("covid_deaths_usafacts.csv", low_memory=False)

# Drop unnecessary columns
df_deaths = df_deaths.drop(columns=["countyFIPS", "StateFIPS"], errors="ignore")

# Rename columns for consistency
df_deaths = df_deaths.rename(columns={"County Name": "County", "State": "State"})

# Standardize county names (remove "County" and extra spaces)
df_deaths["County"] = df_deaths["County"].str.replace(" County", "").str.strip()

# Remove rows where County is "Statewide Unallocated"
df_deaths = df_deaths[df_deaths["County"] != "Statewide Unallocated"]

# Reshape from wide format to long format
df_deaths = df_deaths.melt(id_vars=["State", "County"], var_name="Date", value_name="Deaths")

# Convert Date column to datetime format
df_deaths["Date"] = pd.to_datetime(df_deaths["Date"], format="%Y-%m-%d")

# Extract Year, Month, and Day
df_deaths["Year"] = df_deaths["Date"].dt.year
df_deaths["Month"] = df_deaths["Date"].dt.month
df_deaths["Day"] = df_deaths["Date"].dt.day

# Drop the original Date column
df_deaths = df_deaths.drop(columns=["Date"])

# Set multi-index
df_deaths = df_deaths.set_index(["State", "County", "Year", "Month", "Day"])

# Show processed table
df_deaths.head()

Deaths
State County  Year Month Day        
AL    Autauga 2020 1     22        0
      Baldwin 2020 1     22        0
      Barbour 2020 1     22        0
      Bibb    2020 1     22        0
      Blount  2020 1     22        0

In [5]:
# Merge cases and deaths on multi-index
df_combined = df_cases.merge(df_deaths, on=["State", "County", "Year", "Month", "Day"], how="outer")

# Show merged dataset
df_combined.head()

Cases  Deaths
State County                 Year Month Day               
AK    Aleutians East Borough 2020 1     22       0       0
                                        23       0       0
                                        24       0       0
                                        25       0       0
                                        26       0       0

In [6]:
# Merge df_combined (Cases + Deaths) with df_vax (Vaccination Data)
df_final = df_combined.merge(df_vax, on=["State", "County", "Year", "Month", "Day"], how="outer")

# Show final dataset
df_final.head()

Cases  Deaths  \
State County                 Year Month Day                  
AK    Aleutians East Borough 2020 1     22     0.0     0.0   
                                        23     0.0     0.0   
                                        24     0.0     0.0   
                                        25     0.0     0.0   
                                        26     0.0     0.0   

                                             Series_Complete_Yes  \
State County                 Year Month Day                        
AK    Aleutians East Borough 2020 1     22                   NaN   
                                        23                   NaN   
                                        24                   NaN   
                                        25                   NaN   
                                        26                   NaN   

                                             Series_Complete_Pop_Pct  \
State County                 Year Month Day                            
AK    Aleutians East Borough 2020 1     22                       NaN   
                                        23                       NaN   
                                        24                       NaN   
                                        25                       NaN   
                                        26                       NaN   

                                             Booster_Doses_Vax_Pct  
State County                 Year Month Day                         
AK    Aleutians East Borough 2020 1     22                     NaN  
                                        23                     NaN  
                                        24                     NaN  
                                        25                     NaN  
                                        26                     NaN

In [17]:
import gzip
import csv
#df_final.to_csv("covid_dataset.csv", index=True)
df_final.head()

Cases  Deaths  \
State County                 Year Month Day                  
AK    Aleutians East Borough 2020 1     22     0.0     0.0   
                                        23     0.0     0.0   
                                        24     0.0     0.0   
                                        25     0.0     0.0   
                                        26     0.0     0.0   

                                             Series_Complete_Yes  \
State County                 Year Month Day                        
AK    Aleutians East Borough 2020 1     22                   NaN   
                                        23                   NaN   
                                        24                   NaN   
                                        25                   NaN   
                                        26                   NaN   

                                             Series_Complete_Pop_Pct  \
State County                 Year Month Day                            
AK    Aleutians East Borough 2020 1     22                       NaN   
                                        23                       NaN   
                                        24                       NaN   
                                        25                       NaN   
                                        26                       NaN   

                                             Booster_Doses_Vax_Pct  
State County                 Year Month Day                         
AK    Aleutians East Borough 2020 1     22                     NaN  
                                        23                     NaN  
                                        24                     NaN  
                                        25                     NaN  
                                        26                     NaN

In [18]:

with gzip.open("covid_dataset.csv.gz", 'wt', newline='') as compressed_file:
    csv_writer = csv.writer(compressed_file)
    csv_writer.writerow(df_final.columns)  # Write header
    csv_writer.writerows(df_final.values)